In [ ]:
from hmftpy.hmf import do_hmft, do_hmft_2
from hmftpy.plaquettes.triangular import plaq12, plaq12A, plaq12B
from hmftpy.operators import initialize_mf_hamiltonian, inner_hamiltonian, periodic_hamiltonian
from quspin.basis import spin_basis_1d
import numpy as np
from tqdm import tqdm
from hmftpy.plaquettes.util import plot_plaq
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.linspace(0,.25*np.pi, 100), np.tan(np.linspace(0,.25*np.pi, 100)))

In [ ]:
def plot_plaq(plaq):
    R1 = plaq['Rs'][0]
    R2 = plaq['Rs'][1]
    Rs = [np.zeros(2), R1, R2, -R1, -R2, R1-R2, -R1+R2, R1+R2, -R1-R2]
    for R in Rs:
        outline_rs = [plaq['rs'][o]+R for o in plaq['outline']]
        plt.plot([r[0] for r in outline_rs],
                 [r[1] for r in outline_rs], color='gray', zorder=0)
        plt.fill([r[0] for r in outline_rs],
                 [r[1] for r in outline_rs], color='lightgray', zorder=-1)
        for i, r in enumerate(plaq['rs']):
            ri = r + R
            plt.scatter(*ri, color='black', marker='.')
            plt.text(*ri, i, clip_on=True)
plt.figure(figsize=(4,4), dpi=200)
plot_plaq(plaq12)
plt.xlim(-2, 4)
plt.ylim(-2, 4)
plt.xticks([])
plt.yticks([])
plt.show()

plt.figure(figsize=(4,4), dpi=200)
plot_plaq(plaq12A)
plt.xlim(-2, 4)
plt.ylim(-2, 4)
plt.xticks([])
plt.yticks([])
plt.show()

plt.figure(figsize=(4,4), dpi=200)
plot_plaq(plaq12B)
plt.xlim(-2, 4)
plt.ylim(-2, 4)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
def mf3(plaq):
    mf = {'x': np.zeros(12),
          'y': np.zeros(12),
          'z': np.zeros(12)}
    theta = np.random.rand()*2*np.pi
    if plaq == plaq12:
        SL3 = [[0,5,7,11],
               [1,6,8,3],
               [2,4,9,10]]
    else:
        SL3 = [[0,5,6,11],
               [1,3,7,9],
               [2,4,8,10]]
    # some orientation with 120^o relative in the x-y plane
    mf['x'][SL3[0]] = 0.5*np.cos(theta)
    mf['y'][SL3[0]] = 0.5*np.sin(theta)
    mf['x'][SL3[1]] = 0.5*np.cos(theta+2*np.pi/3)
    mf['y'][SL3[1]] = 0.5*np.sin(theta+2*np.pi/3) 
    mf['x'][SL3[2]] = 0.5*np.cos(theta+4*np.pi/3)
    mf['y'][SL3[2]] = 0.5*np.sin(theta+4*np.pi/3) 
    return mf

def mf4(plaq):
    mf = {'x': np.zeros(12),
          'y': np.zeros(12),
          'z': np.zeros(12)}
    if plaq == plaq12:
        SL4 = [[0,2,8], [1,7,9], [3,5,10], [4,6,11]]
    else:
        SL4 = [[0,2,7], [1,6,8], [3,10,5], [4,9,11]]
    theta = np.random.rand()*2*np.pi
    mf['z'][SL4[0]] = 0.5
    mf['z'][SL4[1]] = 0.5
    mf['z'][SL4[2]] = -0.5
    mf['z'][SL4[3]] = -0.5
    return mf

In [ ]:
steps = 50
basis = spin_basis_1d(12, pauli=0)
thetas = np.linspace(0, .15*np.pi, steps)
interactions = {'n_bonds': {'xx': 1., 'yy': 1., 'zz': 1.},
                'nn_bonds': {'xx': 1., 'yy': 1., 'zz': 1.}}

ET = np.zeros((steps, 2))
EA = np.zeros((steps, 2))
EB = np.zeros((steps, 2))
cvgT = np.zeros((steps, 2))
cvgA = np.zeros((steps, 2))
cvgB = np.zeros((steps, 2))
vT = None
vA = None
vB = None

In [ ]:
mfT = mf3(plaq12)
mfA = mf3(plaq12A)
mfB = mfA
for i, theta in enumerate(tqdm(thetas)):
    J1 = np.cos(theta)
    J2 = np.sin(theta)
    if i == 0:
        mf_cvg = True
    else:
        mf_cvg = False
    interactions = {'n_bonds': {'xx': J1, 'yy': J1, 'zz': J1},
                    'nn_bonds': {'xx': J2, 'yy': J2, 'zz': J2}}
    ET[i,0], vT, mfT, cvgT[i,0] = do_hmft(plaq12, interactions, basis, mf0=mfT, v0=vT, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=100)
    EA[i,0], vA, mfA, cvgA[i,0] = do_hmft(plaq12A, interactions, basis, mf0=mfA, v0=vA, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=100)
    EB[i,0], vB, _, cvgB[i,0] = do_hmft(plaq12B, interactions, basis, mf0=mfA, v0=vA, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=100)


In [ ]:
mfT = mf4(plaq12)
mfA = None
mfB = None
for i, theta in enumerate(tqdm(thetas[::-1])):
    J1 = np.cos(theta)
    J2 = np.sin(theta)
    if i == 0:
        mf_cvg = True
    else:
        mf_cvg = False
    j = steps-1-i
    interactions = {'n_bonds': {'xx': J1, 'yy': J1, 'zz': J1},
                    'nn_bonds': {'xx': J2, 'yy': J2, 'zz': J2}}
    ET[j,1], vT, mfT, cvgT[j,1] = do_hmft(plaq12, interactions, basis, mf0=mfT, v0=vT, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=400)
    EA[j,1], vA, mfA, cvgA[j,1] = do_hmft(plaq12A, interactions, basis, mf0=mfA, v0=vA, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=400)
    EB[j,1], vB, _, cvgB[j,1] = do_hmft(plaq12B, interactions, basis, mf0=mfA, v0=vA, lanczos_tol=10**-12, hmft_tol=10**-9,
                                          mf_cvg=mf_cvg, max_iter=400)

In [ ]:
eT = np.zeros((steps, 2))
eA = np.zeros((steps, 2))
eB = np.zeros((steps, 2))

for i, theta in enumerate(tqdm(thetas)):
    J1 = np.cos(theta)
    J2 = np.sin(theta)
    interactions = {'n_bonds': {'xx': J1, 'yy': J1, 'zz': J1},
                    'nn_bonds': {'xx': J2, 'yy': J2, 'zz': J2}}
    
    hT = inner_hamiltonian(plaq12, interactions, basis)
    e, v = hT.eigsh(k=1, which='SA')
    eT[i,0] = e[0]
    hT = periodic_hamiltonian(plaq12, interactions, basis)
    e, v = hT.eigsh(k=1, which='SA')
    eT[i,1] = e[0]
    
    hA = inner_hamiltonian(plaq12A, interactions, basis)
    e, v = hA.eigsh(k=1, which='SA')
    eA[i,0] = e[0]
    hA = periodic_hamiltonian(plaq12A, interactions, basis)
    e, v = hA.eigsh(k=1, which='SA')
    eA[i,1] = e[0]
    
    hB = inner_hamiltonian(plaq12B, interactions, basis)
    e, v = hB.eigsh(k=1, which='SA')
    eB[i,0] = e[0]
    hB = periodic_hamiltonian(plaq12B, interactions, basis)
    e, v = hB.eigsh(k=1, which='SA')
    eB[i,1] = e[0]

In [ ]:
J2s = np.tan(thetas)

plt.figure(figsize=(5,4), dpi=200)

cvgT = np.array(cvgT, dtype=bool)
plt.scatter(J2s[cvgT[:,0]], ET[:,0][cvgT[:,0]], marker='.', label='TF')
plt.scatter(J2s[cvgT[:,1]], ET[:,1][cvgT[:,1]], marker='.', label='TR')

cvgA = np.array(cvgA, dtype=bool)
plt.scatter(J2s[cvgA[:,0]], EA[:,0][cvgA[:,0]], marker='.', label='AF')
plt.scatter(J2s[cvgA[:,1]], EA[:,1][cvgA[:,1]], marker='.', label='AR')

cvgB = np.array(cvgB, dtype=bool)
plt.scatter(J2s[cvgB[:,0]], EB[:,0][cvgB[:,0]], marker='.', label='BF')
plt.scatter(J2s[cvgB[:,1]], EB[:,1][cvgB[:,1]], marker='.', label='BR')
plt.legend()

In [ ]:
J2s

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
plt.plot(J2s, np.min([np.min(ET, axis=1),
                      eT[:,0]], axis=0), label='Triangular')
plt.plot(J2s, np.min([np.min(EA, axis=1),
                      eA[:,0]], axis=0), label='Parallelogram symmetric')
plt.plot(J2s, np.min([np.min(EB, axis=1),
                      eB[:,0]], axis=0), label='Parallelogram symmetry-breaking')
plt.plot(J2s, eT[:,0], label='Triangular OBC', ls='--')
plt.plot(J2s, eA[:,0], label='Parallelogram OBC', ls='--')
# plt.plot(J2s, eB[:,0], label='Asym parallelogram OBC', ls='--')

# plt.plot(J2s, eT[:,1], label='Triangular PBC', ls='--')
# plt.plot(J2s, eA[:,1], label='Sym parallelogram PBC', ls=':')
# plt.plot(J2s, eB[:,1], label='Asym parallelogram PBC', ls='--')

plt.legend(ncol=2)

# Observables